In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import glob
import numpy as np
import plotly.io as pio
from sklearn.decomposition import PCA

# Open Plotly charts in a separate browser window/tab
pio.renderers.default = "browser"

# Folder containing the data
folder = Path(r"C:\Users\phuynh\Projects\robotray\from_hugo\2026_02_09_nephe")

# Get all voltage CSV files
csv_files = sorted(glob.glob(str(folder / "*Voltage*.csv")))
print(f"Found {len(csv_files)} voltage files")

# Group files by voltage type
voltage_groups = {
    'MiningHighVoltage': [],
    'MiningLowVoltage': [],
    'SoilHighVoltage': [],
    'SoilMidVoltage': [],
    'SoilLowVoltage': []
}

for csv_file in csv_files:
    filename = Path(csv_file).name
    for voltage_type in voltage_groups.keys():
        if voltage_type in filename:
            # Extract test number (first 6 digits)
            test_num = filename[:6]
            voltage_groups[voltage_type].append({
                'file': csv_file,
                'test_num': test_num,
                'filename': filename
            })
            break

# Print counts
for voltage_type, files in voltage_groups.items():
    print(f"{voltage_type}: {len(files)} files")

# Create 5 subplots stacked vertically
fig = make_subplots(
    rows=5, cols=1,
    subplot_titles=(
        "Mining High Voltage",
        "Mining Low Voltage",
        "Soil High Voltage",
        "Soil Mid Voltage",
        "Soil Low Voltage"
    ),
    vertical_spacing=0.05,
    x_title="Energy (keV)",
    y_title="Intensity (CPS)"
)

# Process each voltage type
voltage_types = ['MiningHighVoltage', 'MiningLowVoltage', 'SoilHighVoltage', 'SoilMidVoltage', 'SoilLowVoltage']

for idx, voltage_type in enumerate(voltage_types):
    row = idx + 1
    files_info = voltage_groups[voltage_type]
    
    if not files_info:
        continue
    
    # Load all data for this voltage type
    all_data = []
    for file_info in files_info:
        df = pd.read_csv(file_info['file'])
        if 'Energy (keV)' in df.columns and 'Intensity (CPS)' in df.columns:
            df['test_num'] = file_info['test_num']
            df['filename'] = file_info['filename']
            all_data.append(df)
    
    if not all_data:
        continue
    
    # Merge all data on Energy to align them
    merged = all_data[0][['Energy (keV)', 'Intensity (CPS)']].copy()
    merged = merged.rename(columns={'Intensity (CPS)': f'intensity_0'})
    
    for i, df in enumerate(all_data[1:], start=1):
        df_temp = df[['Energy (keV)', 'Intensity (CPS)']].copy()
        df_temp = df_temp.rename(columns={'Intensity (CPS)': f'intensity_{i}'})
        merged = merged.merge(df_temp, on='Energy (keV)', how='inner')
    
    # PCA-based anomaly detection
    energy = merged['Energy (keV)']
    intensity_cols = [col for col in merged.columns if col.startswith('intensity_')]
    intensity_matrix = merged[intensity_cols].values
    
    # 1) Normalize each curve (each row is one spectrum)
    X = intensity_matrix.T  # Transpose so each row is a spectrum
    X_normalized = (X - X.mean(axis=1, keepdims=True)) / (X.std(axis=1, keepdims=True) + 1e-8)
    
    # 2) PCA with enough components to keep 95% variance
    pca = PCA(n_components=0.95)
    X_pca = pca.fit_transform(X_normalized)
    X_recon = pca.inverse_transform(X_pca)
    
    # 3) Reconstruction error for each spectrum
    errors = np.linalg.norm(X_normalized - X_recon, axis=1)
    
    # 4) Flag top 3% as anomalies
    k = max(1, int(0.03 * len(errors)))
    anomaly_indices = set(np.argsort(errors)[-k:])
    
    # Calculate mean for reference line
    mean_intensity = np.mean(intensity_matrix, axis=1)
    
    print(f"{voltage_type}: Detected {len(anomaly_indices)} anomalies out of {len(files_info)} files (top {k})")
    
    # Plot each trace
    for i, file_info in enumerate(files_info):
        df = all_data[i]
        # Get aligned intensity values from merged dataframe
        intensity = merged[f'intensity_{i}'].values
        
        # Check if this spectrum is an anomaly
        is_anomaly = i in anomaly_indices
        
        if is_anomaly:
            # Bold red line for anomalies
            line_color = 'red'
            line_width = 2.5
            opacity = 1.0
        else:
            # Normal gray line
            line_color = 'lightgray'
            line_width = 0.8
            opacity = 0.5
        
        fig.add_trace(
            go.Scatter(
                x=energy,
                y=intensity,
                mode='lines',
                line=dict(color=line_color, width=line_width),
                opacity=opacity,
                name=file_info['test_num'],
                text=[file_info['test_num']] * len(energy),
                hovertemplate='Test: %{text}<br>Energy: %{x:.2f} keV<br>Intensity: %{y:.2f} CPS<extra></extra>',
                showlegend=False
            ),
            row=row, col=1
        )
    
    # Add mean line for reference
    fig.add_trace(
        go.Scatter(
            x=energy,
            y=mean_intensity,
            mode='lines',
            line=dict(color='blue', width=2, dash='dash'),
            name='Mean',
            showlegend=(row == 1),
            legendgroup='stats',
            hovertemplate='Mean<br>Energy: %{x:.2f} keV<br>Intensity: %{y:.2f} CPS<extra></extra>'
        ),
        row=row, col=1
    )
    
    print(f"Processed {voltage_type}: {len(files_info)} files, {len(anomaly_indices)} anomalies flagged")

# Update layout to match plot_snr dimensions
fig.update_layout(
    height=3000,  # 600 * 5 for 5 stacked plots
    width=1600,   # Same width as plot_snr
    title="Voltage Analysis with PCA-based Anomaly Detection (top 3% by reconstruction error)",
    showlegend=True,
    hovermode='closest'
)

# Update axes
for i in range(1, 6):
    fig.update_xaxes(title_text="Energy (keV)", row=i, col=1)
    fig.update_yaxes(title_text="Intensity (CPS)", row=i, col=1)

fig.show()

print("\nPlot complete!")

Found 750 voltage files
MiningHighVoltage: 150 files
MiningLowVoltage: 150 files
SoilHighVoltage: 150 files
SoilMidVoltage: 150 files
SoilLowVoltage: 150 files
MiningHighVoltage: Detected 4 anomalies out of 150 files (top 4)
Processed MiningHighVoltage: 150 files, 4 anomalies flagged
MiningLowVoltage: Detected 4 anomalies out of 150 files (top 4)
Processed MiningLowVoltage: 150 files, 4 anomalies flagged
SoilHighVoltage: Detected 4 anomalies out of 150 files (top 4)
Processed SoilHighVoltage: 150 files, 4 anomalies flagged
SoilMidVoltage: Detected 4 anomalies out of 150 files (top 4)
Processed SoilMidVoltage: 150 files, 4 anomalies flagged
SoilLowVoltage: Detected 4 anomalies out of 150 files (top 4)
Processed SoilLowVoltage: 150 files, 4 anomalies flagged

Plot complete!
